* * # In this notebook, simple RNN will be used to predict the closing price of Bitcoin

* Any feedback is welcome!!

reference: Deep learning with Python (some lines of code are extracted and modified from this book)
1. https://www.manning.com/books/deep-learning-with-python

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()
# Input data files are available in the "../input/" directory.


from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
#7日間の移動平均を予測するようにしたい。。。
train = pd.read_csv("../input/bitcoindata/bitcoin_price_Training - MA.csv")
test = pd.read_csv("../input/bitcoindata/bitcoin_price_Test - MA.csv")

In [ ]:
train = train[::-1]
test = test[::-1]

In [ ]:
train.head()

In [ ]:
#x is input, y is teach data.
x_train = train['Close'].values.astype('float32')
y_train = train['Next7daysMA'].values.astype('float32')
x_test = test['Close'].values.astype('float32')
y_test = test['Next7daysMA'].values.astype('float32')

# Make sure to standardize the data before feeding it to neural nets

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# reshape data to scale the point
x_train = x_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
x_test = x_test.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [ ]:
scaler = StandardScaler()
x_train_n = scaler.fit_transform(x_train)
y_train_n = scaler.fit_transform(y_train)
x_test_n = scaler.transform(x_test)
y_test_n = scaler.transform(y_test)

In [ ]:
print(x_train_n.shape)
print(y_train_n.shape)
print(x_test_n.shape)
print(y_test_n.shape)

**Hyperparameters:**

lookback: how many points (days) shoud be used as inputs to predict the future value

delay: how many points should be predicted

Ex: In this example, we use previous prices for 24 days as input  to predict the prices for future 7 days

The reason why I chose thses values are:

lookback: 24 is the value chosen by autoregressive model (https://www.kaggle.com/kentata/time-series-data-exploration)

delay: test data has 7 points(daily values) so prediction and true values wiill be possible


In [ ]:
lookback = 24
step = 1
delay = 7
batch_size = 128
train_gen = generator(train_n, lookback=lookback, delay=delay,
    min_index=0, max_index=1000, shuffle=True, step=step,
batch_size=batch_size)
val_gen = generator(train_n, lookback=lookback, delay=delay,
    min_index=1001, max_index=None, step=step, batch_size=batch_size)
test_gen = generator(test_n, lookback=lookback, delay=delay,
    min_index=0, max_index=None, step=step, batch_size=batch_size)
# This is how many steps to draw from `val_gen` in order to see the whole validation set:
val_steps = (len(train_n) - 1001 - lookback) // batch_size
# This is how many steps to draw from `test_gen` in order to see the whole test set:
test_steps = (len(test_n) - lookback) // batch_size

In [ ]:
# reproducibility (make sure each time training is occurred, the result will be the same)
np.random.seed(7)

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

In [ ]:
model = Sequential()
model.add(layers.GRU(32,
                     dropout=0.2,
                     recurrent_dropout=0.2,
                     input_shape=(None, train_n.shape[-1])))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
model.summary()

In [ ]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=40,
                              validation_data=val_gen,
                              validation_steps=val_steps)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))
plt.figure()
plt.plot(epochs, loss, 'blue', label='train loss')
plt.plot(epochs, val_loss, 'orange', label='test loss')
plt.title('Training and validation loss')
plt.legend()

Important to note: Overfitting. Further hyperparamter tuing is necessary (strong drop-out, regularization)

In [ ]:
train_re = train_n.reshape(-1,1,1)
pred = model.predict(train_re)

Since we scaled the data, it is necessary to scale back to data in original units to plot the data

In [ ]:
pred = scaler.inverse_transform(pred)

In [ ]:
plt.plot(range(len(train_re)), train, label='train')
plt.plot(range(len(train_re)), pred, label='prediction')
plt.legend()
plt.title("Prediction on training data")

# Important to note: prediction does not capture the steep increase of price from 1400 days (prediction performs well unitl 1400 days). This might be due to the abrupt change which is hard to predict